## A Monte Carlo simulation technique to determine the optimal portfolio 

In [1]:
import pandas as pd 
import numpy as np 
import yfinance as yf 
import matplotlib.pyplot as plt 
%matplotlib inline 
import matplotlib.dates as mdates 

import warnings 
warnings.filterwarnings('ignore') 

In [2]:
start_date = '2020-01-01'
end_date = '2023-12-30'
# tickers = ['AMD','BAC','CRM','ABBV','CVX','COST','MRK','HD','JNJ','MA','PG','V','UNH','XOM','TSLA',
#            'JPM','LLY','AVGO','BRK-B','GOOG','META','AMZN','AAPL','NVDA','MSFT'] 

tickers = ['AMD','TSLA','GOOG','AMZN'] 

In [3]:
data = pd.DataFrame() 
for tick in tickers:
    data[f'{tick}'] = yf.download(tick,start=start_date,end=end_date)['Adj Close']  

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
data.head() 

,AMD,TSLA,GOOG,AMZN
Date,,,,
2020-01-02,49.099998,28.684000,68.290787,94.900497
2020-01-03,48.599998,29.534000,67.955666,93.748497
2020-01-06,48.389999,30.102667,69.631264,95.143997
2020-01-07,48.250000,31.270666,69.587814,95.343002
2020-01-08,47.830002,32.809334,70.136192,94.598503


In [5]:
data.isna().sum().sum()  

0

#### Calculate daily return rates

In [6]:
returns = data.pct_change().dropna() 
returns.head() 

,AMD,TSLA,GOOG,AMZN
Date,,,,
2020-01-03,-0.010183,0.029633,-0.004907,-0.012139
2020-01-06,-0.004321,0.019255,0.024657,0.014886
2020-01-07,-0.002893,0.038801,-0.000624,0.002092
2020-01-08,-0.008705,0.049205,0.007880,-0.007809
2020-01-09,0.023834,-0.021945,0.011044,0.004799


#### Forecast returns using Historical Data (use ARIMA or LSTM)

In [7]:
from statsmodels.tsa.arima.model import ARIMA

In [8]:
def forecast_returns(series):
    model = ARIMA(series,order=(1,0,0))
    model_fit = model.fit() 
    forecast = model_fit.forecast(steps=1)
    return forecast.values[0] 

Forecast next preiod return for each stock

In [9]:
returns.columns 

Index(['AMD', 'TSLA', 'GOOG', 'AMZN'], dtype='object')

In [10]:
forecast_returns(returns['AMD'])  

d:\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py

0.0025750257820617187

In [11]:
predicted_returns = {stock: forecast_returns(returns[stock]) for stock in tickers}
predicted_returns 

d:\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
d:\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py

{'AMD': 0.0025750257820617187,
 'TSLA': 0.0032405239345557656,
 'GOOG': 0.001292235302672232,
 'AMZN': 0.001145090562233717}

#### Construct Investment Portfolio

Assume equal investment in each stock with a total of $10,000

* Initial Investment

In [12]:
total_investment = 10000
investment_per_stock = total_investment/len(tickers)

* Investment Allocation

In [13]:
investment_allocation = {stock: investment_per_stock for stock in tickers}
investment_allocation 

{'AMD': 2500.0, 'TSLA': 2500.0, 'GOOG': 2500.0, 'AMZN': 2500.0}

#### Calculate Predicted Return or Loss

In [14]:
predicted_dollar_returns = {stock: investment_allocation[stock]*predicted_returns[stock] for stock in tickers} 
predicted_dollar_returns 

{'AMD': 6.4375644551542965,
 'TSLA': 8.101309836389413,
 'GOOG': 3.23058825668058,
 'AMZN': 2.8627264055842923}

#### Simulate Returns using Monte Carlo Simulation

Monte Carlo simulation is used to generate possible future outcomes. Normal distribution of returns are assumed for simplicity

In [15]:
num_sims = 10000
simulated_returns = {stock: np.random.normal(loc=predicted_returns[stock],scale=returns[stock].std(),size=num_sims) for stock in tickers}

simulated_returns_df = pd.DataFrame(simulated_returns)
simulated_returns_df.head() 

,AMD,TSLA,GOOG,AMZN
0,-0.010732,0.042355,-0.002394,0.015531
1,0.023311,0.062444,-0.007095,-0.002069
2,0.022046,0.020566,0.031291,-0.038186
3,-0.015643,-0.024288,-0.006023,0.019006
4,-0.026987,0.040827,0.023716,0.027680


#### Calculate VaR and Optimize Portfolio

Calculating VaR at 95% confidence level

In [16]:
VaR = simulated_returns_df.quantile(0.05)
VaR 

AMD    -0.053497
TSLA   -0.067742
GOOG   -0.033520
AMZN   -0.038878
Name: 0.05, dtype: float64

Portfolio Optimization using Markowitz Portfolio Theory

In [17]:
import cvxpy as cp 

In [18]:
mean_returns = simulated_returns_df.mean()
mean_returns 

AMD     0.002043
TSLA    0.002828
GOOG    0.001296
AMZN    0.001162
dtype: float64

In [19]:
cov_matrix = simulated_returns_df.cov()
cov_matrix 

,AMD,TSLA,GOOG,AMZN
AMD,1.145042e-03,2.047616e-05,0.000010,3.961324e-07
TSLA,2.047616e-05,1.840829e-03,-0.000005,4.317705e-07
GOOG,9.741208e-06,-4.785826e-06,0.000456,5.277523e-06
AMZN,3.961324e-07,4.317705e-07,0.000005,5.737892e-04


Define optimization problem

In [20]:
weights = cp.Variable(len(tickers)) 
portfolio_return = mean_returns.values @ weights
portfolio_volatility = cp.quad_form(weights,cov_matrix) 
risk_tolerance = 1 
objective = cp.Maximize(portfolio_return-risk_tolerance*portfolio_volatility)
constraints = [cp.sum(weights)==1, weights>=0]
problem = cp.Problem(objective,constraints)
problem.solve() 

0.0017105091535933735

In [21]:
optimal_weights = weights.value
optimal_portfolio = {tickers[i]: optimal_weights[i] for i in range(len(tickers))}
optimal_portfolio 

{'AMD': 0.38098949702144586,
 'TSLA': 0.45221275125117044,
 'GOOG': 0.15747237855344293,
 'AMZN': 0.009325373173940775}

## Hybrid Model

In [22]:
mean_returns

AMD     0.002043
TSLA    0.002828
GOOG    0.001296
AMZN    0.001162
dtype: float64

In [23]:
cov_matrix

,AMD,TSLA,GOOG,AMZN
AMD,1.145042e-03,2.047616e-05,0.000010,3.961324e-07
TSLA,2.047616e-05,1.840829e-03,-0.000005,4.317705e-07
GOOG,9.741208e-06,-4.785826e-06,0.000456,5.277523e-06
AMZN,3.961324e-07,4.317705e-07,0.000005,5.737892e-04


In [24]:
n_assets = len(tickers)
weights = cp.Variable(n_assets)
risk_aversion = 0.5
target_return = 0.11 

In [25]:
portfolio_ret = cp.sum(weights @ mean_returns)
portfolio_var = cp.quad_form(weights,cov_matrix)
utility = portfolio_ret-(risk_aversion/2)*portfolio_var

In [26]:
constraints = [cp.sum(weights)==1,weights>=0,portfolio_ret>=target_return]

In [27]:
sector_constraints = [weights[0]<=0.3,weights[2]<=0.3]

In [28]:
constraints+=sector_constraints

In [29]:
problem = cp.Problem(cp.Maximize(utility),constraints)
problem.solve() 

-inf

In [30]:
# Get the optimal weights
optimal_weights = weights.value

# Print results
print("Optimal Weights for Each Asset:")
for i in range(n_assets):
    print(f"Asset {i+1}: {optimal_weights[i]:.4f}") 

Optimal Weights for Each Asset:


TypeError: 'NoneType' object is not subscriptable